In [1]:
import pandas as pd
import numpy as np
import h2o

# Machine Learning - Using h2o Auto ML

In [2]:
full_table = pd.read_csv('./data/processed_results/osc_df')

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_231"; Java(TM) SE Runtime Environment (build 1.8.0_231-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from /Users/nickparker/anaconda3/envs/DistributedComputing/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmpbdspipni
  JVM stdout: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmpbdspipni/h2o_nickparker_started_from_python.out
  JVM stderr: /var/folders/kj/kg5zhyt51z7__kkfrysh9lv00000gn/T/tmpbdspipni/h2o_nickparker_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.2
H2O cluster version age:,2 days
H2O cluster name:,H2O_from_python_nickparker_itfi5a
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


### Train/Test Split

First Year of Existance:
- gg_drama 1943
- gg_comedy 1951
- pga 1989
- bafta 1960
- dga 1948
- sag 1995
- cannes 1970

In [4]:
# I pick a min_year where the awards shows will be relevant
min_year = 1995

# Leftover code from designing the split
# # 80/20 Split (random)
# table_sized = full_table.loc[(full_table['year'] >= min_year)]
# rand_rows = np.random.choice(table_sized.shape[0], table_sized.shape[0])
# split = int(0.8 * len(rand_rows))
# train = table_sized.iloc[rand_rows[:split]]
# valid = table_sized.iloc[rand_rows[split:]]

# # 80/20 Split (sequential)
# year_range = 2019 - min_year
# train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
# valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

# # Final Prediction
# train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
# test = full_table.loc[(full_table['year'] == 2019)]

# train.to_csv('./data/processed_results/train.csv', index = False)
# valid.to_csv('./data/processed_results/valid.csv', index = False)

# H2O's Auto ML

## Pick best model by validating with the first 80% of the data

In [5]:
# 80/20 Split (sequential) Excluding 2019
year_range = 2019 - min_year
train = full_table.loc[((full_table['year'] <= 2019 - 0.2 * year_range) & (full_table['year'] >= min_year))]
valid = full_table.loc[((full_table['year'] > 2019 - 0.2 * year_range) & (full_table['year'] < 2019))]

In [6]:
print('training set contains:', train.shape[0], 'movies')
print('validation set contains:', valid.shape[0], 'movies')

training set contains: 127 movies
validation set contains: 36 movies


In [7]:
train.columns

Index(['Unnamed: 0', 'year', 'film', 'wiki', 'winner', 'Nominations',
       'Oscar_win', 'nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy',
       'winner_gg_comedy', 'nom_pga', 'winner_pga', 'nom_bafta',
       'winner_bafta', 'nom_dga', 'winner_dga', 'nom_sag', 'winner_sag',
       'nom_cannes', 'winner_cannes'],
      dtype='object')

In [8]:
from h2o.automl import H2OAutoML, get_leaderboard

# Import a sample binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(valid)

# Identify predictors and response
predictors = ['year','Nominations','nom_gg_drama', 'winner_gg_drama', 'nom_gg_comedy', 'winner_gg_comedy',
       'nom_pga', 'winner_pga', 'nom_bafta', 'winner_bafta', 'nom_dga', 'winner_dga',
        'nom_sag', 'winner_sag', 'nom_cannes', 'winner_cannes']

x = predictors
y = 'Oscar_win'

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

# AutoML Leaderboard
lb = aml.leaderboard

# Optionally edd extra model information to the leaderboard
lb = get_leaderboard(aml, extra_columns='ALL')

# Print all rows (instead of default 10 rows)
lb.head(rows=lb.nrows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |██████████
12:31:20.631: Skipping training of model GBM_5_AutoML_20200123_123107 due to exception: water.exceptions.H2OModelBuilderIllegalArgumentException: Illegal argument(s) for GBM model: GBM_5_AutoML_20200123_123107.  Details: ERRR on field: _min_rows: The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 127.0.


██████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
DeepLearning_grid__3_AutoML_20200123_123107_model_1,0.942722,0.600262,0.774146,0.133199,0.287284,0.0825319,8357,0.119379
DeepLearning_grid__2_AutoML_20200123_123107_model_1,0.918239,0.758569,0.680352,0.171159,0.328702,0.108045,4927,0.098853
XGBoost_grid__1_AutoML_20200123_123107_model_4,0.914645,0.296144,0.545711,0.113657,0.304747,0.092871,195,0.022506
StackedEnsemble_AllModels_AutoML_20200123_123107,0.901168,0.262895,0.601087,0.109164,0.277252,0.0768689,154,0.551175
XGBoost_grid__1_AutoML_20200123_123107_model_1,0.898023,0.353486,0.50837,0.118374,0.332499,0.110555,234,0.044145
StackedEnsemble_BestOfFamily_AutoML_20200123_123107,0.871743,0.243171,0.541832,0.109164,0.264445,0.0699312,352,0.137756
XGBoost_grid__1_AutoML_20200123_123107_model_2,0.866128,0.36574,0.44159,0.141509,0.339137,0.115014,208,0.063443
XRT_1_AutoML_20200123_123107,0.85894,0.358339,0.459112,0.228212,0.328076,0.107634,51,0.041165
DeepLearning_grid__1_AutoML_20200123_123107_model_1,0.851752,0.572082,0.503527,0.151842,0.340839,0.116171,2363,0.027832
DeepLearning_1_AutoML_20200123_123107,0.849506,0.497906,0.413588,0.185085,0.370573,0.137325,61,0.029727


## Once best model is picked, retrain that on the entire dataset

In [9]:
top_model = aml.leader
top_model

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid__3_AutoML_20200123_123107_model_1


Status of Neuron Layers: predicting Oscar_win, 2-class classification, bernoulli distribution, CrossEntropy loss, 84,202 weights/biases, 999.6 KB, 97,790 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,16,Input,20,,,,,,,,,
1,,2,200,RectifierDropout,40,0,0,0.0470822,0.0849866,0,0.0060908,0.259766,0.248076,0.216169
2,,3,200,RectifierDropout,40,0,0,0.236978,0.257536,0,-0.0156103,0.126213,0.806937,0.760979
3,,4,200,RectifierDropout,40,0,0,0.189004,0.272257,0,-0.0253725,0.0942923,0.778657,0.103901
4,,5,2,Softmax,,0,0,0.0398194,0.0818308,0,0.000645878,0.342716,0.12369,0.0371031




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.0032728130781669203
RMSE: 0.05720850529568938
LogLoss: 0.012318017292116191
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 0.9523809523809523
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8269310842255554: 


,,0,1,Error,Rate
0,0,106.0,0.0,0.0,(0.0/106.0)
1,1,0.0,21.0,0.0,(0.0/21.0)
2,Total,106.0,21.0,0.0,(0.0/127.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.76123077,0.42672607,0.9230769,0.9230769,0.96,0.0,1.0
1,auc,0.92543644,0.08659743,0.8068182,0.9166667,0.9782609,NaN,1.0
2,aucpr,0.399747,0.34954986,0.47182018,0.83550125,0.29166666,NaN,0.0
3,err,0.03945514,0.03762493,0.07692308,0.07692308,0.04,0.0034295414,0.0
4,err_count,1.25,0.9574271,2.0,2.0,1.0,NaN,0.0
5,f0point5,0.67860544,0.39762497,0.75,0.9285714,0.71428573,1.6990445E-4,1.0
6,f1,0.86964285,0.114934914,0.75,0.9285714,0.8,NaN,1.0
7,f2,-Infinity,NaN,0.75,0.9285714,0.90909094,-Infinity,1.0
8,lift_top_group,9.174035,10.076389,6.5,1.8571428,12.5,0.01303474,25.0
9,logloss,0.5934597,0.42600715,1.1154395,0.8496642,0.40474215,None,0.5974527



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-01-23 12:32:48,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-01-23 12:32:48,38.612 sec,6135 obs/sec,10.0,1,1270.0,0.260785,0.360085,0.507227,0.947889,0.757660,6.047619,0.062992
2,,2020-01-23 12:32:53,43.690 sec,11076 obs/sec,460.0,46,58420.0,0.113747,0.035815,0.906251,0.999102,0.948146,6.047619,0.007874
3,,2020-01-23 12:32:56,46.741 sec,11762 obs/sec,770.0,77,97790.0,0.057209,0.012318,0.976286,1.000000,0.952381,6.047619,0.000000



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,winner_dga,1.000000,1.000000,0.090521
1,Nominations,0.934925,0.934925,0.084631
2,winner_sag,0.853640,0.853640,0.077273
3,year,0.838386,0.838386,0.075892
4,nom_pga,0.717904,0.717904,0.064985
5,winner_bafta,0.703369,0.703369,0.063670
6,winner_pga,0.697011,0.697011,0.063094
7,winner_cannes,0.674687,0.674687,0.061073
8,nom_bafta,0.653698,0.653698,0.059173
9,nom_sag,0.634151,0.634151,0.057404


In [10]:
# Re-split data on min_year to 2019
train = full_table.loc[((full_table['year'] < 2019) & (full_table['year'] >= min_year))]
test = full_table.loc[(full_table['year'] == 2019)]

# Import a binary outcome train/test set into H2O
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [11]:
top_model.train(x=x, y=y, training_frame=train)

preds = top_model.predict(test)

preds

deeplearning Model Build progress: |██████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1
0,0.999997,3.13478e-06
0,0.965335,0.034665
0,0.999736,0.000264017
0,0.999816,0.000183518
0,0.999993,7.28585e-06
0,0.999967,3.26213e-05
0,0.866514,0.133486
0,0.554461,0.445539
0,0.989254,0.0107462


## Predict the winner

In [12]:
test['pred'] = preds['predict']
test['probA'] = preds['p1']
test_pd = test.as_data_frame(use_pandas=True)

In [13]:
test_pd[['film','probA']].sort_values('probA', ascending = False)

,film,probA
7,Once Upon a Time in Hollywood,0.445539
6,1917 (2019 film),0.133486
1,The Irishman,0.034665
8,Parasite (2019 film),0.010746
2,Jojo Rabbit,0.000264
3,Joker (2019 film),0.000184
5,Marriage Story,0.000033
4,Little Women (2019 film),0.000007
0,Ford v Ferrari,0.000003


In [14]:
test_pd.T #The DGA's (1/26) and BAFTA's (2/2) Haven't come out yet

,0,1,2,3,4,5,6,7,8
Unnamed: 0,564,565,566,567,568,569,570,571,572
year,2019,2019,2019,2019,2019,2019,2019,2019,2019
film,Ford v Ferrari,The Irishman,Jojo Rabbit,Joker (2019 film),Little Women (2019 film),Marriage Story,1917 (2019 film),Once Upon a Time in Hollywood,Parasite (2019 film)
wiki,/wiki/Ford_v_Ferrari,/wiki/The_Irishman,/wiki/Jojo_Rabbit,/wiki/Joker_(2019_film),/wiki/Little_Women_(2019_film),/wiki/Marriage_Story,/wiki/1917_(2019_film),/wiki/Once_Upon_a_Time_in_Hollywood,/wiki/Parasite_(2019_film)
winner,False,False,False,False,False,False,False,False,False
Nominations,4,10,6,11,6,6,10,10,6
Oscar_win,0,0,0,0,0,0,0,0,0
nom_gg_drama,0,1,0,1,0,1,1,0,0
winner_gg_drama,0,0,0,0,0,0,1,0,0
nom_gg_comedy,0,0,1,0,0,0,0,1,0
